# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [179]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [180]:

print(os.getcwd())
file_path_list = []
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'


for root, dirs, files in os.walk(filepath):
    

    file_path_list.append(glob.glob(os.path.join(root,'*')))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [181]:

full_data_rows_list = [] 
    

for f in file_path_list[0]:


    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)       
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [182]:

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [183]:


from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [184]:


try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class':'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [185]:

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [186]:
#drop tables
def drop_tables(table_name):
    
    try:
        session.execute(f"DROP table IF EXISTS {table_name}")
        print(f"Droped {table_name} table successfuly!")
    except Exception as e:
        print(f" Error: dropping table {table_name}")
        print(e)

print(drop_tables("users_info"))
print(drop_tables("songs_info"))
print(drop_tables("song_playlist_session"))

Droped users_info table successfuly!
None
Droped songs_info table successfuly!
None
Droped song_playlist_session table successfuly!
None


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [187]:

query = "CREATE TABLE IF NOT EXISTS songs_info"
query = query + "(session_id int, itemInSession int, artist text, song text, song_length float, PRIMARY KEY(session_id, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [188]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_info (session_id, itemInSession, artist, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),  int(line[3]), line[0], line[9],  float(line[5])))

In [189]:
query2 = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query2 = query2 + """(artist text, song text, firstName text, lastName text, user_id int, session_id int, itemInSession int, PRIMARY KEY ((user_id, session_id), itemInSession))"""
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [190]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (user_id, session_id, artist, \
            song, firstName, lastName, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

In [191]:
query3 = "CREATE TABLE IF NOT EXISTS users_info"
query3 = query3 + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [192]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_info (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [193]:

a_query = "select * from songs_info"
try:
    rows = session.execute(a_query)
except Exception as e:
    print(e)

In [194]:

a_query = "select * from song_playlist_session"
try:
    rows = session.execute(a_query)
except Exception as e:
    print(e)

In [195]:

a_query = "select * from users_info"
try:
    rows = session.execute(a_query)
except Exception as e:
    print(e)    

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [196]:

a_query = """select artist, song, song_length
from songs_info
where session_id = 338  AND itemInSession = 4  
"""
try:
    rows = session.execute(a_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)     

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


Query 1: In order to get the artist, song, and song length for session 338 and iteminsession 4, I thought it would be best to set the primary keys as session_id and iteminsession. This would give us all the results that meet this particular session_id and iteminsession.

In [197]:
b_query = """select artist, song
FROM song_playlist_session
WHERE user_id = 10 AND session_id = 182
"""
try:
    rows = session.execute(b_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)            

Row(artist='Down To The Bone', song="Keep On Keepin' On")
Row(artist='Three Drives', song='Greece 2000')
Row(artist='Sebastien Tellier', song='Kilometer')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


Query 2: To get the artist name, song, and user I believe it would be best to set the primary key as user_id and session_id while using iteminSession as the clustering column.

In [198]:
c_query = """select first_name, last_name
from users_info
WHERE song = 'All Hands Against His Own'"""
try:
    rows = session.execute(c_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


Query 3: To get all user names who listed to a particular song "All hands against his Own" I thought it would be best to use song, first name and last name as the primary keys. This will return all the users that meet this criteria. 

### Drop the tables before closing out the sessions

In [199]:
# print(drop_tables("users"))
# print(drop_tables("songs"))
# print(drop_tables("artists"))

### Close the session and cluster connection¶

In [200]:
# session.shutdown()
# cluster.shutdown()